## 1. 词向量
自然语言需要数学化才能够被计算机认识和计算。数学化的方法有很多，最简单的方法是为每个词分配一个编号，这种方法已经有多种应用，但是依然存在一个缺点：不能表示词与词的关系。  

词向量是这样的一种向量\[0.1, -3.31, 83.37, 93.0, -18.37, ……\]，每一个词对应一个向量，词义相近的词，他们的词向量距离也会越近(欧氏距离、夹角余弦)  

词向量有一个优点，就是维度一般较低，一般是50维或100维，这样可以避免维度灾难，也更容易使用深度学习  
### 1.1 训练词向量
语言模型表达的实际就是已知前n-1个词的前提下，预测第n个词的概率。  

词向量的训练是一种无监督学习，也就是没有标注数据，给我n篇文章，我就可以训练出词向量。  

基于三层神经网络构建n-gram语言模型(词向量顺带着就算出来了)的基本思路：  
![](Images/基于三层神经网络构建n-gram语言模型.png)  
最下面的w是词，其上面的C(w)是词向量，词向量一层也就是神经网络的输入层(第一层)，这个输入层是一个(n-1)×m的矩阵，其中n-1是词向量数目，m是词向量维度  

第二层(隐藏层)是就是普通的神经网络，以H为权重，以tanh为激活函数  

第三层(输出层)有|V|个节点，|V|就是词表的大小，输出以U为权重，以softmax作为激活函数以实现归一化，最终就是输出可能是某个词的概率。  

另外，神经网络中有一个技巧就是增加一个从输入层到输出层的直连边(线性变换)，这样可以提升模型效果，这个变换矩阵设为W  

$假设C(w)就是输入的x，那么y的计算公式就是y = b + Wx + Utanh(d+Hx)$  

这个模型里面需要训练的有这么几个变量：$C、H、U、W$。利用梯度下降法训练之后得出的C就是生成词向量所用的矩阵，$C(w$)表示的就是我们需要的词向量  

上面是讲解词向量如何“顺带”训练出来的，然而真正有用的地方在于这个词向量如何进一步应用。  <br/>  
### 1.2 词向量的应用
第一种应用是找同义词。具体应用案例就是google的word2vec工具，通过训练好的词向量，指定一个词，可以返回和它$cos$距离最相近的词并排序。  

第二种应用是词性标注和语义角色标注任务。具体使用方法是：把词向量作为神经网络的输入层，通过前馈网络和卷积网络完成。  

第三种应用是句法分析和情感分析任务。具体使用方法是：把词向量作为递归神经网络的输入。  

第四种应用是命名实体识别和短语识别。具体使用方法是：把词向量作为扩展特征使用。  

另外词向量有一个非常特别的现象：$C(king)-C(queue)≈C(man)-C(woman)$，这里的减法就是向量逐维相减，换个表达方式就是：$C(king)-C(man)+C(woman)$和它最相近的向量就是$C(queue)$，这里面的原理其实就是：语义空间中的线性关系。基于这个结论相信会有更多奇妙的功能出现  

## 2. word2vec的实现原理
词向量是将深度学习应用到NLP的根基，word2vec是google的文本挖掘深度学习工具，如今使用最广泛最简单有效的词向量训练工具，那么它的实现原理是怎样的呢？本文将从原理出发来介绍word2vec   <br/>  
问你这样一个问题：如果你大脑有很多记忆单元，让你记住一款白色奥迪Q7运动型轿车，你会用几个记忆单元？你也许会用一个记忆单元，因为这样最节省你的大脑。那么我们再让你记住一款小型灰色雷克萨斯，你会怎么办？显然你会用另外一个记忆单元来记住它。那么如果让你记住所有的车，你要耗费的记忆单元就不再是那么少了，这种表示方法叫做localist representation。这时你可能会换另外一种思路：我们用几个记忆单元来分别识别大小、颜色、品牌等基础信息，这样通过这几个记忆单元的输出，我们就可以表示出所有的车型了。这种表示方法叫做distributed representation，词向量就是一种用distributed representation表示的向量  
<br/>  
### 2.1 localist representation与distributed representation
localist representation中文释义是稀疏表达，典型的案例就是one hot vector，也就是这样的一种向量表示：

\[1, 0, 0, 0, 0, 0……]表示成年男子

[0, 1, 0, 0, 0, 0……]表示成年女子

[0, 0, 1, 0, 0, 0……]表示老爷爷

[0, 0, 0, 1, 0, 0……]表示老奶奶

[0, 0, 0, 0, 1, 0……]表示男婴

[0, 0, 0, 0, 0, 1……]表示女婴

……

每一类型用向量中的一维来表示  
<br/> 
而distributed representation中文释义是分布式表达，上面的表达方式可以改成：

性别 老年 成年 婴儿

[1,       0,      1,      0]表示成年男子

[0,       0,      1,      0]表示成年女子

[1,       1,      0,      0]表示老爷爷

[0,       1,      0,      0]表示老奶奶

[1,       0,      0,      1]表示男婴

[0,       0,      0,      1]表示女婴

如果我们想表达男童和女童，只需要增加一个特征维度即可

### 2.2 word embedding
翻译成中文叫做词嵌入，这里的embedding来源于范畴论，在范畴论中称为morphism(态射)，态射表示两个数学结构中保持结构的一种过程抽象，比如“函数”、“映射”，他们都是表示一个域和另一个域之间的某种关系。  

范畴论中的嵌入(态射)是要保持结构的，而word embedding表示的是一种“降维”的嵌入，通过降维避免维度灾难，降低计算复杂度，从而更易于在深度学习中应用。  

理解了distributed representation和word embedding的概念，我们就初步了解了word2vec的本质，它其实是通过distributed representation的表达方式来表示词，而且通过降维的word embedding来减少计算量的一种方法  

## 3. word2vec中的神经网络
word2vec中做训练主要使用的是神经概率语言模型，这需要掌握一些基础知识  
在word2vec中使用的最重要的两个模型是CBOW和Skip-gram模型，下面我们分别来介绍这两种模型  
<br/> 
### 3.1 CBOW模型
CBOW全称是Continuous Bag-of-Words Model，是在已知当前词的上下文的前提下预测当前词  
![](Images/CBOW模型.png)  
CBOW模型的神经网络结构设计如下：  

* 输入层：词w的上下文一共2c个词的词向量  

* 投影层：将输入层的2c个向量做求和累加  

* 输出层：一个霍夫曼树，其中叶子节点是语料中出现过的词，权重是出现的次数   
神经网络模型把首尾相接改成了求和累加，这样减少了维度；去掉了隐藏层，这样减少了计算量；输出层由softmax归一化运算改成了霍夫曼树；这一系列修改对训练的性能有很大提升，而效果不减，这是独到之处。  
<br/>  
### 3.2 基于霍夫曼树的Hierarchical Softmax技术
上面的CBOW输出层为什么要建成一个霍夫曼树呢？因为我们是要基于训练语料得到每一个可能的w的概率。那么具体怎么得到呢  
<br/> 
## 4. Skip-gram模型
Skip-gram全称是Continuous Skip-gram Model，是在已知当前词的情况下预测上下文  
![](Images/Skip-gram模型.png) 
Skip-gram模型的神经网络结构设计如下：  

* 输入层：w的词向量v(w)  

* 投影层：依然是v(w)，就是一个形式  

* 输出层：和CBOW一样的霍夫曼树  

后面的推导公式和CBOW大同小异，其中θ和v(w)的更新公式除了把符号名从x改成了v(w)之外完全一样